In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [2]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='12G')

3.26.0.2
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_222"; OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1~deb9u1-b10); OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpf9hwf70e
  JVM stdout: /tmp/tmpf9hwf70e/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpf9hwf70e/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,27 days
H2O cluster name:,H2O_from_python_unknownUser_6inr6x
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
X_train=h2o.import_file("../input/house-prices-advanced-regression-techniques/train.csv")
X_test=h2o.import_file("../input/house-prices-advanced-regression-techniques/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
for f in X_train.columns:
    if X_train[f].columns_by_type(coltype='categorical'): 
        X_train[f]=X_train[f].asfactor()

In [5]:
for f in X_test.columns:
    if X_test[f].columns_by_type(coltype='categorical'): 
        X_test[f]=X_test[f].asfactor()

In [6]:
X_train = X_train.fillna((method="forward",axis=0,maxlen=1)
X_test = X_test.fillna((method="forward",axis=0,maxlen=1)

SyntaxError: invalid syntax (<ipython-input-6-398f68191d6f>, line 1)

In [7]:
a=X_train.drop(['SalePrice'], axis=1)
x=a.columns[1:]
y='SalePrice'

In [8]:
%%time
aml = H2OAutoML(max_models=30, seed=23, max_runtime_secs=30000)
aml.train(x=x, y=y, training_frame=X_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 4.41 s, sys: 520 ms, total: 4.93 s
Wall time: 12min 52s


In [9]:
%%time
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 10.3 ms


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid_1_AutoML_20190823_050900_model_1,7.30082e+08,27020,7.30082e+08,15492.8,0.13008
GBM_grid_1_AutoML_20190823_050900_model_4,7.62235e+08,27608.6,7.62235e+08,15965.2,0.132491
XGBoost_2_AutoML_20190823_050900,7.89608e+08,28100,7.89608e+08,15775.5,0.131364
XGBoost_grid_1_AutoML_20190823_050900_model_9,8.01814e+08,28316.3,8.01814e+08,15979.1,0.133424
XGBoost_grid_1_AutoML_20190823_050900_model_5,8.07243e+08,28412,8.07243e+08,16167.6,0.134804
GBM_3_AutoML_20190823_050900,8.13421e+08,28520.5,8.13421e+08,16000.9,0.133698
GBM_2_AutoML_20190823_050900,8.15422e+08,28555.6,8.15422e+08,16381.4,0.136654
GBM_4_AutoML_20190823_050900,8.18389e+08,28607.5,8.18389e+08,16324.6,0.135803
XGBoost_grid_1_AutoML_20190823_050900_model_7,8.30242e+08,28813.9,8.30242e+08,15770.2,0.133038
XGBoost_1_AutoML_20190823_050900,8.32949e+08,28860.9,8.32949e+08,15793.2,0.132981


In [10]:
%%time
aml.leader

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 19.5 ms
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_20190823_050900_model_1

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,88133.0,9.0,14.0,13.01,24.0,79.0,65.15




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 161689325.96764252
RMSE: 12715.711775895305
MAE: 6300.096792059076
RMSLE: 0.05968794530406137
Mean Residual Deviance: 161689325.96764252

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 730081866.0308942
RMSE: 27020.027128611364
MAE: 15492.847374139037
RMSLE: 0.1300796764166922
Mean Residual Deviance: 730081866.0308942
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,15492.848,1449.6783,13702.31,14860.666,15243.847,19453.488,14203.926
mean_residual_deviance,730081860.0000000,276284992.0000000,394725376.0000000,643180670.0000000,601244480.0000000,1492900480.0000000,518358272.0000000
mse,730081860.0000000,276284992.0000000,394725376.0000000,643180670.0000000,601244480.0000000,1492900480.0000000,518358272.0000000
r2,0.8895591,0.0284340,0.9265866,0.8984937,0.9011716,0.8115692,0.9099746
residual_deviance,730081860.0000000,276284992.0000000,394725376.0000000,643180670.0000000,601244480.0000000,1492900480.0000000,518358272.0000000
rmse,26230.908,4583.7363,19867.697,25361.008,24520.287,38638.07,22767.482
rmsle,0.1287858,0.0129412,0.1269947,0.1139625,0.1308635,0.1619562,0.1101519


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-08-23 05:13:30,6.952 sec,0.0,79415.2918861,57434.7702758,6306788585.3492975
,2019-08-23 05:13:31,7.043 sec,5.0,64441.4255212,45797.0140197,4152697323.2052960
,2019-08-23 05:13:31,7.138 sec,10.0,52904.5510289,36834.3261130,2798891519.5645838
,2019-08-23 05:13:31,7.228 sec,15.0,44201.6191349,29875.4150150,1953783134.1500762
,2019-08-23 05:13:31,7.323 sec,20.0,37486.4256663,24503.3798587,1405232109.2354522
---,---,---,---,---,---,---
,2019-08-23 05:13:32,8.488 sec,80.0,14425.9173366,7103.9174952,208107091.0029863
,2019-08-23 05:13:32,8.573 sec,85.0,13976.9488002,6852.0530153,195355097.7627993
,2019-08-23 05:13:32,8.656 sec,90.0,13517.0810649,6625.4337302,182711480.5156260
,2019-08-23 05:13:32,8.739 sec,95.0,13095.0082228,6456.0130966,171479240.3561597



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
OverallQual,19440622108672.0000000,1.0,0.2114805
ExterQual,10115323265024.0000000,0.5203189,0.1100373
GrLivArea,9966324809728.0000000,0.5126546,0.1084165
Neighborhood,8954364035072.0000000,0.4606007,0.0974081
GarageCars,7506320949248.0000000,0.3861153,0.0816559
---,---,---,---
Heating,0.0,0.0,0.0
PoolArea,0.0,0.0,0.0
PoolQC,0.0,0.0,0.0
MiscFeature,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [11]:
%%time
y_preds = aml.predict(X_test)
y_preds.head(10)
# y_preds['p1'].as_data_frame().values.flatten().shape

gbm prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'MSZoning' has levels not trained on: [NA]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Utilities' has levels not trained on: [NA]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior1st' has levels not trained on: [NA]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior2nd' has levels not trained on: [NA]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'KitchenQual' has levels not trained on: [NA]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Functional' has levels not trained on: [NA]
  warnings.warn(w)
/opt/conda/lib/python3.6/sit

CPU times: user 84 ms, sys: 12 ms, total: 96 ms
Wall time: 327 ms


predict
124362
155485
178708
191970
182710
179510
173528
173679
180914
126616


In [12]:
%%time
sample_submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_submission['SalePrice'] = y_preds[:].as_data_frame().values
sample_submission.to_csv('H2O_AutoML_submission_1.csv', index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

CPU times: user 1.95 s, sys: 396 ms, total: 2.34 s
Wall time: 5.34 s
